# List of Model Available 

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [ ]:
#import timm
#model = timm.create_model('adv_inception_v3', pretrained=True)
#model.eval()


## Imagenet - a Data

In [1]:
import numpy as np
import sklearn.metrics as sk
recall_level_default = 0.95

def calib_err(confidence, correct, p='2', beta=100):
    # beta is target bin size
    idxs = np.argsort(confidence)
    confidence = confidence[idxs]
    correct = correct[idxs]
    bins = [[i * beta, (i + 1) * beta] for i in range(len(confidence) // beta)]
    bins[-1] = [bins[-1][0], len(confidence)]

    cerr = 0
    total_examples = len(confidence)
    for i in range(len(bins) - 1):
        bin_confidence = confidence[bins[i][0]:bins[i][1]]
        bin_correct = correct[bins[i][0]:bins[i][1]]
        num_examples_in_bin = len(bin_confidence)

        if num_examples_in_bin > 0:
            difference = np.abs(np.nanmean(bin_confidence) - np.nanmean(bin_correct))

            if p == '2':
                cerr += num_examples_in_bin / total_examples * np.square(difference)
            elif p == '1':
                cerr += num_examples_in_bin / total_examples * difference
            elif p == 'infty' or p == 'infinity' or p == 'max':
                cerr = np.maximum(cerr, difference)
            else:
                assert False, "p must be '1', '2', or 'infty'"

    if p == '2':
        cerr = np.sqrt(cerr)

    return cerr


def aurra(confidence, correct):
    conf_ranks = np.argsort(confidence)[::-1]  # indices from greatest to least confidence
    rra_curve = np.cumsum(np.asarray(correct)[conf_ranks])
    rra_curve = rra_curve / np.arange(1, len(rra_curve) + 1)  # accuracy at each response rate
    return np.mean(rra_curve)


def soft_f1(confidence, correct):
    wrong = 1 - correct

    # # the incorrectly classified samples are our interest
    # # so they make the positive class
    # tp_soft = np.sum((1 - confidence) * wrong)
    # fp_soft = np.sum((1 - confidence) * correct)
    # fn_soft = np.sum(confidence * wrong)

    # return 2 * tp_soft / (2 * tp_soft + fn_soft + fp_soft)
    return 2 * ((1 - confidence) * wrong).sum()/(1 - confidence + wrong).sum()


def tune_temp(logits, labels, binary_search=True, lower=0.2, upper=5.0, eps=0.0001):
    logits = np.array(logits)

    if binary_search:
        import torch
        import torch.nn.functional as F

        logits = torch.FloatTensor(logits)
        labels = torch.LongTensor(labels)
        t_guess = torch.FloatTensor([0.5*(lower + upper)]).requires_grad_()

        while upper - lower > eps:
            if torch.autograd.grad(F.cross_entropy(logits / t_guess, labels), t_guess)[0] > 0:
                upper = 0.5 * (lower + upper)
            else:
                lower = 0.5 * (lower + upper)
            t_guess = t_guess * 0 + 0.5 * (lower + upper)

        t = min([lower, 0.5 * (lower + upper), upper], key=lambda x: float(F.cross_entropy(logits / x, labels)))
    else:
        import cvxpy as cx

        set_size = np.array(logits).shape[0]

        t = cx.Variable()

        expr = sum((cx.Minimize(cx.log_sum_exp(logits[i, :] * t) - logits[i, labels[i]] * t)
                    for i in range(set_size)))
        p = cx.Problem(expr, [lower <= t, t <= upper])

        p.solve()   # p.solve(solver=cx.SCS)
        t = 1 / t.value

    return t


def print_measures(rms, aurra_metric, mad, sf1, method_name='Baseline'):
    print('\t\t\t\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(100 * rms))
    print('AURRA (%): \t\t\t{:.2f}'.format(100 * aurra))
    # print('MAD Calib Error (%): \t\t{:.2f}'.format(100 * mad))
    # print('Soft F1 Score (%):   \t\t{:.2f}'.format(100 * sf1))


def show_calibration_results(confidence, correct, method_name='Baseline'):

    print('\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(
        100 * calib_err(confidence, correct, p='2')))

    print('AURRA (%): \t\t\t{:.2f}'.format(
        100 * aurra(confidence, correct)))

    # print('MAD Calib Error (%): \t\t{:.2f}'.format(
    #     100 * calib_err(confidence, correct, p='1')))

    # print('Soft F1-Score (%): \t\t{:.2f}'.format(
    #     100 * soft_f1(confidence, correct)))

def fpr_and_fdr_at_recall(y_true, y_score, recall_level=recall_level_default, pos_label=None):
    classes = np.unique(y_true)
    if (pos_label is None and
            not (np.array_equal(classes, [0, 1]) or
                     np.array_equal(classes, [-1, 1]) or
                     np.array_equal(classes, [0]) or
                     np.array_equal(classes, [-1]) or
                     np.array_equal(classes, [1]))):
        raise ValueError("Data is not binary and pos_label is not specified")
    elif pos_label is None:
        pos_label = 1.

    # make y_true a boolean vector
    y_true = (y_true == pos_label)

    # sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]

    # y_score typically has many tied values. Here we extract
    # the indices associated with the distinct values. We also
    # concatenate a value for the end of the curve.
    distinct_value_indices = np.where(np.diff(y_score))[0]
    threshold_idxs = np.r_[distinct_value_indices, y_true.size - 1]

    # accumulate the true positives with decreasing threshold
    tps = stable_cumsum(y_true)[threshold_idxs]
    fps = 1 + threshold_idxs - tps      # add one because of zero-based indexing

    thresholds = y_score[threshold_idxs]

    recall = tps / tps[-1]

    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)      # [last_ind::-1]
    recall, fps, tps, thresholds = np.r_[recall[sl], 1], np.r_[fps[sl], 0], np.r_[tps[sl], 0], thresholds[sl]

    cutoff = np.argmin(np.abs(recall - recall_level))

    return fps[cutoff] / (np.sum(np.logical_not(y_true)))   # , fps[cutoff]/(fps[cutoff] + tps[cutoff])

def get_measures(_pos, _neg, recall_level=recall_level_default):
    pos = np.array(_pos[:]).reshape((-1, 1))
    neg = np.array(_neg[:]).reshape((-1, 1))
    examples = np.squeeze(np.vstack((pos, neg)))
    labels = np.zeros(len(examples), dtype=np.int32)
    labels[:len(pos)] += 1

    auroc = sk.roc_auc_score(labels, examples)
    aupr = sk.average_precision_score(labels, examples)
    fpr = fpr_and_fdr_at_recall(labels, examples, recall_level)

    return auroc, aupr, fpr


def print_measures_old(auroc, aupr, fpr, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}'.format(int(100 * recall_level), 100 * fpr))
    print('AUROC: \t{:.2f}'.format(100 * auroc))
    print('AUPR:  \t{:.2f}'.format(100 * aupr))


def print_measures_with_std(aurocs, auprs, fprs, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}\t+/- {:.2f}'.format(int(100 * recall_level), 100 * np.mean(fprs), 100 * np.std(fprs)))
    print('AUROC: \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(aurocs), 100 * np.std(aurocs)))
    print('AUPR:  \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(auprs), 100 * np.std(auprs)))


def get_and_print_results(out_score, in_score, num_to_avg=1):

    aurocs, auprs, fprs = [], [], []
    #for _ in range(num_to_avg):
    #    out_score = get_ood_scores(ood_loader)
    measures = get_measures(out_score, in_score)
    aurocs.append(measures[0]); auprs.append(measures[1]); fprs.append(measures[2])

    auroc = np.mean(aurocs); aupr = np.mean(auprs); fpr = np.mean(fprs)
    #auroc_list.append(auroc); aupr_list.append(aupr); fpr_list.append(fpr)

    #if num_to_avg >= 5:
    #    print_measures_with_std(aurocs, auprs, fprs, method_name='Ours')
    #else:
    #    print_measures(auroc, aupr, fpr, method_name='Ours')
    return auroc, aupr, fpr 

# Imagenet - a: Dataloader

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as trn
import torchvision.transforms.functional as trnF
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import numpy as np

thousand_k_to_200 = {0: -1, 1: -1, 2: -1, 3: -1, 4: -1, 5: -1, 6: 0, 7: -1, 8: -1, 9: -1, 10: -1, 11: 1, 12: -1, 13: 2, 14: -1, 15: 3, 16: -1, 17: 4, 18: -1, 19: -1, 20: -1, 21: -1, 22: 5, 23: 6, 24: -1, 25: -1, 26: -1, 27: 7, 28: -1, 29: -1, 30: 8, 31: -1, 32: -1, 33: -1, 34: -1, 35: -1, 36: -1, 37: 9, 38: -1, 39: 10, 40: -1, 41: -1, 42: 11, 43: -1, 44: -1, 45: -1, 46: -1, 47: 12, 48: -1, 49: -1, 50: 13, 51: -1, 52: -1, 53: -1, 54: -1, 55: -1, 56: -1, 57: 14, 58: -1, 59: -1, 60: -1, 61: -1, 62: -1, 63: -1, 64: -1, 65: -1, 66: -1, 67: -1, 68: -1, 69: -1, 70: 15, 71: 16, 72: -1, 73: -1, 74: -1, 75: -1, 76: 17, 77: -1, 78: -1, 79: 18, 80: -1, 81: -1, 82: -1, 83: -1, 84: -1, 85: -1, 86: -1, 87: -1, 88: -1, 89: 19, 90: 20, 91: -1, 92: -1, 93: -1, 94: 21, 95: -1, 96: 22, 97: 23, 98: -1, 99: 24, 100: -1, 101: -1, 102: -1, 103: -1, 104: -1, 105: 25, 106: -1, 107: 26, 108: 27, 109: -1, 110: 28, 111: -1, 112: -1, 113: 29, 114: -1, 115: -1, 116: -1, 117: -1, 118: -1, 119: -1, 120: -1, 121: -1, 122: -1, 123: -1, 124: 30, 125: 31, 126: -1, 127: -1, 128: -1, 129: -1, 130: 32, 131: -1, 132: 33, 133: -1, 134: -1, 135: -1, 136: -1, 137: -1, 138: -1, 139: -1, 140: -1, 141: -1, 142: -1, 143: 34, 144: 35, 145: -1, 146: -1, 147: -1, 148: -1, 149: -1, 150: 36, 151: 37, 152: -1, 153: -1, 154: -1, 155: -1, 156: -1, 157: -1, 158: -1, 159: -1, 160: -1, 161: -1, 162: -1, 163: -1, 164: -1, 165: -1, 166: -1, 167: -1, 168: -1, 169: -1, 170: -1, 171: -1, 172: -1, 173: -1, 174: -1, 175: -1, 176: -1, 177: -1, 178: -1, 179: -1, 180: -1, 181: -1, 182: -1, 183: -1, 184: -1, 185: -1, 186: -1, 187: -1, 188: -1, 189: -1, 190: -1, 191: -1, 192: -1, 193: -1, 194: -1, 195: -1, 196: -1, 197: -1, 198: -1, 199: -1, 200: -1, 201: -1, 202: -1, 203: -1, 204: -1, 205: -1, 206: -1, 207: 38, 208: -1, 209: -1, 210: -1, 211: -1, 212: -1, 213: -1, 214: -1, 215: -1, 216: -1, 217: -1, 218: -1, 219: -1, 220: -1, 221: -1, 222: -1, 223: -1, 224: -1, 225: -1, 226: -1, 227: -1, 228: -1, 229: -1, 230: -1, 231: -1, 232: -1, 233: -1, 234: 39, 235: 40, 236: -1, 237: -1, 238: -1, 239: -1, 240: -1, 241: -1, 242: -1, 243: -1, 244: -1, 245: -1, 246: -1, 247: -1, 248: -1, 249: -1, 250: -1, 251: -1, 252: -1, 253: -1, 254: 41, 255: -1, 256: -1, 257: -1, 258: -1, 259: -1, 260: -1, 261: -1, 262: -1, 263: -1, 264: -1, 265: -1, 266: -1, 267: -1, 268: -1, 269: -1, 270: -1, 271: -1, 272: -1, 273: -1, 274: -1, 275: -1, 276: -1, 277: 42, 278: -1, 279: -1, 280: -1, 281: -1, 282: -1, 283: 43, 284: -1, 285: -1, 286: -1, 287: 44, 288: -1, 289: -1, 290: -1, 291: 45, 292: -1, 293: -1, 294: -1, 295: 46, 296: -1, 297: -1, 298: 47, 299: -1, 300: -1, 301: 48, 302: -1, 303: -1, 304: -1, 305: -1, 306: 49, 307: 50, 308: 51, 309: 52, 310: 53, 311: 54, 312: -1, 313: 55, 314: 56, 315: 57, 316: -1, 317: 58, 318: -1, 319: 59, 320: -1, 321: -1, 322: -1, 323: 60, 324: 61, 325: -1, 326: 62, 327: 63, 328: -1, 329: -1, 330: 64, 331: -1, 332: -1, 333: -1, 334: 65, 335: 66, 336: 67, 337: -1, 338: -1, 339: -1, 340: -1, 341: -1, 342: -1, 343: -1, 344: -1, 345: -1, 346: -1, 347: 68, 348: -1, 349: -1, 350: -1, 351: -1, 352: -1, 353: -1, 354: -1, 355: -1, 356: -1, 357: -1, 358: -1, 359: -1, 360: -1, 361: 69, 362: -1, 363: 70, 364: -1, 365: -1, 366: -1, 367: -1, 368: -1, 369: -1, 370: -1, 371: -1, 372: 71, 373: -1, 374: -1, 375: -1, 376: -1, 377: -1, 378: 72, 379: -1, 380: -1, 381: -1, 382: -1, 383: -1, 384: -1, 385: -1, 386: 73, 387: -1, 388: -1, 389: -1, 390: -1, 391: -1, 392: -1, 393: -1, 394: -1, 395: -1, 396: -1, 397: 74, 398: -1, 399: -1, 400: 75, 401: 76, 402: 77, 403: -1, 404: 78, 405: -1, 406: -1, 407: 79, 408: -1, 409: -1, 410: -1, 411: 80, 412: -1, 413: -1, 414: -1, 415: -1, 416: 81, 417: 82, 418: -1, 419: -1, 420: 83, 421: -1, 422: -1, 423: -1, 424: -1, 425: 84, 426: -1, 427: -1, 428: 85, 429: -1, 430: 86, 431: -1, 432: -1, 433: -1, 434: -1, 435: -1, 436: -1, 437: 87, 438: 88, 439: -1, 440: -1, 441: -1, 442: -1, 443: -1, 444: -1, 445: 89, 446: -1, 447: -1, 448: -1, 449: -1, 450: -1, 451: -1, 452: -1, 453: -1, 454: -1, 455: -1, 456: 90, 457: 91, 458: -1, 459: -1, 460: -1, 461: 92, 462: 93, 463: -1, 464: -1, 465: -1, 466: -1, 467: -1, 468: -1, 469: -1, 470: 94, 471: -1, 472: 95, 473: -1, 474: -1, 475: -1, 476: -1, 477: -1, 478: -1, 479: -1, 480: -1, 481: -1, 482: -1, 483: 96, 484: -1, 485: -1, 486: 97, 487: -1, 488: 98, 489: -1, 490: -1, 491: -1, 492: 99, 493: -1, 494: -1, 495: -1, 496: 100, 497: -1, 498: -1, 499: -1, 500: -1, 501: -1, 502: -1, 503: -1, 504: -1, 505: -1, 506: -1, 507: -1, 508: -1, 509: -1, 510: -1, 511: -1, 512: -1, 513: -1, 514: 101, 515: -1, 516: 102, 517: -1, 518: -1, 519: -1, 520: -1, 521: -1, 522: -1, 523: -1, 524: -1, 525: -1, 526: -1, 527: -1, 528: 103, 529: -1, 530: 104, 531: -1, 532: -1, 533: -1, 534: -1, 535: -1, 536: -1, 537: -1, 538: -1, 539: 105, 540: -1, 541: -1, 542: 106, 543: 107, 544: -1, 545: -1, 546: -1, 547: -1, 548: -1, 549: 108, 550: -1, 551: -1, 552: 109, 553: -1, 554: -1, 555: -1, 556: -1, 557: 110, 558: -1, 559: -1, 560: -1, 561: 111, 562: 112, 563: -1, 564: -1, 565: -1, 566: -1, 567: -1, 568: -1, 569: 113, 570: -1, 571: -1, 572: 114, 573: 115, 574: -1, 575: 116, 576: -1, 577: -1, 578: -1, 579: 117, 580: -1, 581: -1, 582: -1, 583: -1, 584: -1, 585: -1, 586: -1, 587: -1, 588: -1, 589: 118, 590: -1, 591: -1, 592: -1, 593: -1, 594: -1, 595: -1, 596: -1, 597: -1, 598: -1, 599: -1, 600: -1, 601: -1, 602: -1, 603: -1, 604: -1, 605: -1, 606: 119, 607: 120, 608: -1, 609: 121, 610: -1, 611: -1, 612: -1, 613: -1, 614: 122, 615: -1, 616: -1, 617: -1, 618: -1, 619: -1, 620: -1, 621: -1, 622: -1, 623: -1, 624: -1, 625: -1, 626: 123, 627: 124, 628: -1, 629: -1, 630: -1, 631: -1, 632: -1, 633: -1, 634: -1, 635: -1, 636: -1, 637: -1, 638: -1, 639: -1, 640: 125, 641: 126, 642: 127, 643: 128, 644: -1, 645: -1, 646: -1, 647: -1, 648: -1, 649: -1, 650: -1, 651: -1, 652: -1, 653: -1, 654: -1, 655: -1, 656: -1, 657: -1, 658: 129, 659: -1, 660: -1, 661: -1, 662: -1, 663: -1, 664: -1, 665: -1, 666: -1, 667: -1, 668: 130, 669: -1, 670: -1, 671: -1, 672: -1, 673: -1, 674: -1, 675: -1, 676: -1, 677: 131, 678: -1, 679: -1, 680: -1, 681: -1, 682: 132, 683: -1, 684: 133, 685: -1, 686: -1, 687: 134, 688: -1, 689: -1, 690: -1, 691: -1, 692: -1, 693: -1, 694: -1, 695: -1, 696: -1, 697: -1, 698: -1, 699: -1, 700: -1, 701: 135, 702: -1, 703: -1, 704: 136, 705: -1, 706: -1, 707: -1, 708: -1, 709: -1, 710: -1, 711: -1, 712: -1, 713: -1, 714: -1, 715: -1, 716: -1, 717: -1, 718: -1, 719: 137, 720: -1, 721: -1, 722: -1, 723: -1, 724: -1, 725: -1, 726: -1, 727: -1, 728: -1, 729: -1, 730: -1, 731: -1, 732: -1, 733: -1, 734: -1, 735: -1, 736: 138, 737: -1, 738: -1, 739: -1, 740: -1, 741: -1, 742: -1, 743: -1, 744: -1, 745: -1, 746: 139, 747: -1, 748: -1, 749: 140, 750: -1, 751: -1, 752: 141, 753: -1, 754: -1, 755: -1, 756: -1, 757: -1, 758: 142, 759: -1, 760: -1, 761: -1, 762: -1, 763: 143, 764: -1, 765: 144, 766: -1, 767: -1, 768: 145, 769: -1, 770: -1, 771: -1, 772: -1, 773: 146, 774: 147, 775: -1, 776: 148, 777: -1, 778: -1, 779: 149, 780: 150, 781: -1, 782: -1, 783: -1, 784: -1, 785: -1, 786: 151, 787: -1, 788: -1, 789: -1, 790: -1, 791: -1, 792: 152, 793: -1, 794: -1, 795: -1, 796: -1, 797: 153, 798: -1, 799: -1, 800: -1, 801: -1, 802: 154, 803: 155, 804: 156, 805: -1, 806: -1, 807: -1, 808: -1, 809: -1, 810: -1, 811: -1, 812: -1, 813: 157, 814: -1, 815: 158, 816: -1, 817: -1, 818: -1, 819: -1, 820: 159, 821: -1, 822: -1, 823: 160, 824: -1, 825: -1, 826: -1, 827: -1, 828: -1, 829: -1, 830: -1, 831: 161, 832: -1, 833: 162, 834: -1, 835: 163, 836: -1, 837: -1, 838: -1, 839: 164, 840: -1, 841: -1, 842: -1, 843: -1, 844: -1, 845: 165, 846: -1, 847: 166, 848: -1, 849: -1, 850: 167, 851: -1, 852: -1, 853: -1, 854: -1, 855: -1, 856: -1, 857: -1, 858: -1, 859: 168, 860: -1, 861: -1, 862: 169, 863: -1, 864: -1, 865: -1, 866: -1, 867: -1, 868: -1, 869: -1, 870: 170, 871: -1, 872: -1, 873: -1, 874: -1, 875: -1, 876: -1, 877: -1, 878: -1, 879: 171, 880: 172, 881: -1, 882: -1, 883: -1, 884: -1, 885: -1, 886: -1, 887: -1, 888: 173, 889: -1, 890: 174, 891: -1, 892: -1, 893: -1, 894: -1, 895: -1, 896: -1, 897: 175, 898: -1, 899: -1, 900: 176, 901: -1, 902: -1, 903: -1, 904: -1, 905: -1, 906: -1, 907: 177, 908: -1, 909: -1, 910: -1, 911: -1, 912: -1, 913: 178, 914: -1, 915: -1, 916: -1, 917: -1, 918: -1, 919: -1, 920: -1, 921: -1, 922: -1, 923: -1, 924: 179, 925: -1, 926: -1, 927: -1, 928: -1, 929: -1, 930: -1, 931: -1, 932: 180, 933: 181, 934: 182, 935: -1, 936: -1, 937: 183, 938: -1, 939: -1, 940: -1, 941: -1, 942: -1, 943: 184, 944: -1, 945: 185, 946: -1, 947: 186, 948: -1, 949: -1, 950: -1, 951: 187, 952: -1, 953: -1, 954: 188, 955: -1, 956: 189, 957: 190, 958: -1, 959: 191, 960: -1, 961: -1, 962: -1, 963: -1, 964: -1, 965: -1, 966: -1, 967: -1, 968: -1, 969: -1, 970: -1, 971: 192, 972: 193, 973: -1, 974: -1, 975: -1, 976: -1, 977: -1, 978: -1, 979: -1, 980: 194, 981: 195, 982: -1, 983: -1, 984: 196, 985: -1, 986: 197, 987: 198, 988: 199, 989: -1, 990: -1, 991: -1, 992: -1, 993: -1, 994: -1, 995: -1, 996: -1, 997: -1, 998: -1, 999: -1}

indices_in_1k = [k for k in thousand_k_to_200 if thousand_k_to_200[k] != -1]

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

test_transform = trn.Compose(
    [trn.Resize(256), trn.CenterCrop(224), trn.ToTensor(), trn.Normalize(mean, std)])

naes = dset.ImageFolder(root="/content/drive/MyDrive/vision_project/imagenet-a", transform=test_transform)
nae_loader = torch.utils.data.DataLoader(naes, batch_size=128, shuffle=True,
                                         num_workers=4, pin_memory=True)

In [5]:
naes

Dataset ImageFolder
    Number of datapoints: 7500
    Root location: /content/drive/MyDrive/vision_project/imagenet-a
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

# Example

In [ ]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to('cpu').numpy()

def get_net_results(model_name, num_cls = 1000):
    confidence = []
    correct = []
    model = timm.create_model(model_name, pretrained=True, num_classes=num_cls)
    net = model
    net.cuda()
    net.eval()

    num_correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(nae_loader):
            data, target = data.cuda(), target.cuda()

            output = net(data)[:,indices_in_1k]

            # accuracy
            pred = output.data.max(1)[1]
            num_correct += pred.eq(target.data).sum().item()

            confidence.extend(to_np(F.softmax(output, dim=1).max(1)[0]).squeeze().tolist())
            pred = output.data.max(1)[1]
            correct.extend(pred.eq(target).to('cpu').numpy().squeeze().tolist())

    return num_correct / len(nae_loader.dataset), confidence.copy(), correct.copy()

In [ ]:
acc, test_confidence, test_correct = get_net_results(model_name='pit_b_224')

print('ImageNet-A Accuracy (%):', round(100*acc, 4))

show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-pit-weights/pit_b_820.pth" to /root/.cache/torch/hub/checkpoints/pit_b_820.pth


ImageNet-A Accuracy (%): 33.8933
				Baseline
RMS Calib Error (%): 		20.75
AURRA (%): 			49.02


# Recipe - 1 Running - CNN Model

In [ ]:
cnn_model_lst1 = ['cspresnet50', 'densenet121', 'dla34', 'dpn68', 'efficientnetv2_rw_m']

In [ ]:
cnn_model_lst2 = ['efficientnet_b0', 'mixnet_l', 'mnasnet_100', 'fbnetc_100', 'spnasnet_100', 'tinynet_a',
                 'ghostnet_100', 'hrnet_w18', 'inception_resnet_v2', 'nasnetalarge', 'pnasnet5large',
                 'res2net50_14w_8s', 'resnest14d', 'resnet10t', 'rexnet_100', 'selecsls42b', 'skresnet18',
                  'tnt_s_patch16_224', 'vgg19', 'ese_vovnet19b_dw', 'xception']

In [ ]:
len(cnn_model_lst2)

21

In [ ]:
cnn_accuracy = []

for name in cnn_model_lst1:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  cnn_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/cspresnet50_ra-d3e8d487.pth" to /root/.cache/torch/hub/checkpoints/cspresnet50_ra-d3e8d487.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/densenet121_ra-50efcf5c.pth" to /root/.cache/torch/hub/checkpoints/densenet121_ra-50efcf5c.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/dla34-2b83ff04.pth" to /root/.cache/torch/hub/checkpoints/dla34-2b83ff04.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnetv2_rw_m_agc-3d90cb1e.pth" to /root/.cache/torch/hub/checkpoints/efficientnetv2_rw_m_agc-3d90cb1e.pth


In [ ]:
cnn_accuracy

['ImageNet-A Accuracy (%) for cspresnet50 model: 6.7067',
 'ImageNet-A Accuracy (%) for densenet121 model: 4.9733',
 'ImageNet-A Accuracy (%) for dla34 model: 3.2133',
 'ImageNet-A Accuracy (%) for dpn68 model: 3.6267',
 'ImageNet-A Accuracy (%) for efficientnetv2_rw_m model: 22.3067']

In [ ]:
cnn_accuracy2 = []

for name in cnn_model_lst2:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  cnn_accuracy2.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mixnet_l-5a9a2ed8.pth" to /root/.cache/torch/hub/checkpoints/mixnet_l-5a9a2ed8.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mnasnet_b1-74cb7081.pth" to /root/.cache/torch/hub/checkpoints/mnasnet_b1-74cb7081.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/fbnetc_100-c345b898.pth" to /root/.cache/torch/hub/checkpoints/fbnetc_100-c345b898.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/spnasnet_100-048bc3f4.pth" to /root/.cache/torch/hub/checkpoints/spnasnet_100-048bc3f4.pth
Downloading: "https://github.com/huawei-noah/CV-Backbones/releases/downloa

In [ ]:
cnn_accuracy2

['ImageNet-A Accuracy (%) for efficientnet_b0 model: 7.0667',
 'ImageNet-A Accuracy (%) for mixnet_l model: 8.4267',
 'ImageNet-A Accuracy (%) for mnasnet_100 model: 2.88',
 'ImageNet-A Accuracy (%) for fbnetc_100 model: 4.0933',
 'ImageNet-A Accuracy (%) for spnasnet_100 model: 3.5733',
 'ImageNet-A Accuracy (%) for tinynet_a model: 7.8933',
 'ImageNet-A Accuracy (%) for ghostnet_100 model: 3.24',
 'ImageNet-A Accuracy (%) for hrnet_w18 model: 5.5067',
 'ImageNet-A Accuracy (%) for inception_resnet_v2 model: 5.72',
 'ImageNet-A Accuracy (%) for nasnetalarge model: 3.4933',
 'ImageNet-A Accuracy (%) for pnasnet5large model: 3.3867',
 'ImageNet-A Accuracy (%) for res2net50_14w_8s model: 5.0533',
 'ImageNet-A Accuracy (%) for resnest14d model: 5.4667',
 'ImageNet-A Accuracy (%) for resnet10t model: 1.88',
 'ImageNet-A Accuracy (%) for rexnet_100 model: 8.8',
 'ImageNet-A Accuracy (%) for selecsls42b model: 4.5333',
 'ImageNet-A Accuracy (%) for skresnet18 model: 2.7467',
 'ImageNet-A Acc

In [ ]:
len(cnn_accuracy2)

21

In [ ]:
cnn_model_lst3 = ['inception_v3', 'nf_resnet50']

In [ ]:
cnn_accuracy3 = []

for name in cnn_model_lst3:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  cnn_accuracy3.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/nf_resnet50_ra2-9f236009.pth" to /root/.cache/torch/hub/checkpoints/nf_resnet50_ra2-9f236009.pth


In [ ]:
cnn_accuracy3

['ImageNet-A Accuracy (%) for inception_v3 model: 5.6533',
 'ImageNet-A Accuracy (%) for nf_resnet50 model: 6.7733']

# Recipe - 2 - Running - Vision Transformer Model

In [ ]:
## vit_large_patch16_384 = An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale

In [ ]:
## divide the model by 4-5 
vit_model_lst = ['beit_base_patch16_224', 'cait_s24_224', 'coatnet_0_rw_224', 'crossvit_base_240', 
                 'deit_base_patch16_224', 'deit3_base_patch16_224', 'efficientformer_l1', 'gcvit_base',
                 'maxvit_rmlp_small_rw_224', 'mobilevit_s', 'mvitv2_base', 'jx_nest_base', 'pit_b_224'
                 'pvt_v2_b0', 'swin_s3_base_224', 'swinv2_cr_small_224', 'tnt_s_patch16_224', 'twins_pcpvt_bas',
                 'visformer_small', 'vit_base_patch32_224', 'volo_d1_224', 'xcit_small_12_p8_224']

In [ ]:
vit_accuracy = []

for name in vit_model_lst:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  vit_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

In [ ]:
vit_accuracy

['ImageNet-A Accuracy (%) for beit_base_patch16_224 model: 3.92',
 'ImageNet-A Accuracy (%) for cait_s24_224 model: 34.4',
 'ImageNet-A Accuracy (%) for coatnet_0_rw_224 model: 16.68',
 'ImageNet-A Accuracy (%) for crossvit_base_240 model: 31.8',
 'ImageNet-A Accuracy (%) for deit_base_patch16_224 model: 27.8533',
 'ImageNet-A Accuracy (%) for deit3_base_patch16_224 model: 40.12',
 'ImageNet-A Accuracy (%) for efficientformer_l1 model: 13.9333',
 'ImageNet-A Accuracy (%) for gcvit_base model: 42.7067']

In [ ]:
vit_model_lst2 = ['maxvit_tiny_rw_224', 'mobilevit_s', 'mvitv2_base', 'jx_nest_base', 'pit_s_224'
                 'pvt_v2_b0', 'swin_s3_base_224', 'swinv2_cr_small_224', 'tnt_s_patch16_224', 'twins_pcpvt_bas',
                 'visformer_small', 'vit_base_patch32_224', 'volo_d1_224', 'xcit_small_12_p8_224']

In [ ]:
vit_accuracy2 = []

for name in vit_model_lst2:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  vit_accuracy2.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

In [ ]:
vit_accuracy2

['ImageNet-A Accuracy (%) for maxvit_tiny_rw_224 model: 23.32',
 'ImageNet-A Accuracy (%) for mobilevit_s model: 0.3333',
 'ImageNet-A Accuracy (%) for mvitv2_base model: 43.0933',
 'ImageNet-A Accuracy (%) for jx_nest_base model: 34.1467']

In [ ]:
vit_model_lst3 = ['pit_b_224', 'pvt_v2_b0', 'swin_s3_base_224', 'swinv2_cr_small_224', 'tnt_s_patch16_224', 'twins_pcpvt_base',
                 'visformer_small', 'vit_base_patch32_224', 'volo_d1_224', 'xcit_small_12_p8_224']

In [ ]:
vit_accuracy3 = []

for name in vit_model_lst3:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  vit_accuracy3.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vt3p-weights/twins_pcpvt_base-e5ecb09b.pth" to /root/.cache/torch/hub/checkpoints/twins_pcpvt_base-e5ecb09b.pth
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vt3p-weights/visformer_small-839e1f5b.pth" to /root/.cache/torch/hub/checkpoints/visformer_small-839e1f5b.pth
Downloading: "https://github.com/sail-sg/volo/releases/download/volo_1/d1_224_84.2.pth.tar" to /root/.cache/torch/hub/checkpoints/d1_224_84.2.pth.tar
Downloading: "https://dl.fbaipublicfiles.com/xcit/xcit_small_12_p8_224.pth" to /root/.cache/torch/hub/checkpoints/xcit_small_12_p8_224.pth


In [ ]:
vit_accuracy3

['ImageNet-A Accuracy (%) for pit_b_224 model: 33.8933',
 'ImageNet-A Accuracy (%) for pvt_v2_b0 model: 4.0533',
 'ImageNet-A Accuracy (%) for swin_s3_base_224 model: 39.4133',
 'ImageNet-A Accuracy (%) for swinv2_cr_small_224 model: 31.5867',
 'ImageNet-A Accuracy (%) for tnt_s_patch16_224 model: 17.1733',
 'ImageNet-A Accuracy (%) for twins_pcpvt_base model: 30.1067',
 'ImageNet-A Accuracy (%) for visformer_small model: 26.0267',
 'ImageNet-A Accuracy (%) for vit_base_patch32_224 model: 11.2533',
 'ImageNet-A Accuracy (%) for volo_d1_224 model: 41.0267',
 'ImageNet-A Accuracy (%) for xcit_small_12_p8_224 model: 40.6933']

# Recipe - 3 - Combination of CNN and Vision Transformer

In [ ]:
combine_model_lst = ['convnext_base', 'convit_base', 'edgenext_base', 'levit_128']

In [ ]:
combine_accuracy = []

for name in combine_model_lst:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  combine_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth
Downloading: "https://dl.fbaipublicfiles.com/convit/convit_base.pth" to /root/.cache/torch/hub/checkpoints/convit_base.pth
Downloading: "https://github.com/mmaaz60/EdgeNeXt/releases/download/v1.2/edgenext_base_usi.pth" to /root/.cache/torch/hub/checkpoints/edgenext_base_usi.pth
Downloading: "https://dl.fbaipublicfiles.com/LeViT/LeViT-128-b88c2750.pth" to /root/.cache/torch/hub/checkpoints/LeViT-128-b88c2750.pth


In [ ]:
combine_accuracy

['ImageNet-A Accuracy (%) for convnext_base model: 35.7867',
 'ImageNet-A Accuracy (%) for convit_base model: 29.6933',
 'ImageNet-A Accuracy (%) for edgenext_base model: 21.8267',
 'ImageNet-A Accuracy (%) for levit_128 model: 11.5067']

# Recipe - 4 - Running MLP Mixer

In [ ]:
mlp_model_lst = ['mixer_b16_224']

In [ ]:
mlp_accuracy = []

for name in mlp_model_lst:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  mlp_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_mixer_b16_224-76587d61.pth" to /root/.cache/torch/hub/checkpoints/jx_mixer_b16_224-76587d61.pth


In [ ]:
mlp_accuracy

['ImageNet-A Accuracy (%) for mixer_b16_224 model: 5.7867']

# 5.    Recipe - 5 - Mobilenet

In [ ]:
mobilenet_model_lst = ['mobilenetv3_large_100']

In [ ]:
mobilenet_accuracy = []

for name in mobilenet_model_lst:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  mobilenet_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /root/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


In [ ]:
mobilenet_accuracy

['ImageNet-A Accuracy (%) for mobilenetv3_large_100 model: 4.5867']

# 6.  Recipe - 6 - Others

In [ ]:
others_model_lst = ['regnetv_040', 'sequencer2d_l']

In [ ]:
others_accuracy = []

for name in others_model_lst:
  acc, test_confidence, test_correct = get_net_results(model_name= name)
  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
  others_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth


In [ ]:
others_accuracy

['ImageNet-A Accuracy (%) for regnetv_040 model: 26.48',
 'ImageNet-A Accuracy (%) for sequencer2d_l model: 35.36']

# 7.  Recipe - 7 - Perceiver

# Cross Checking Model Name

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [ ]:
model = timm.create_model('dpn92')
model